In an American put, an early exercise of the option may be optimal and no
closed-form formula exists for the price.

The original formula was for a call option, where the payoff at expiry is $\max(S_T - c, 0)$. For a put option, the payoff is inverted. The holder profits if the stock price $S_T$ is below the strike price $c$. Therefore, the boundary condition at the final time step $n$ becomes
\begin{equation}
    V(n, j) = \max(c - S(n, j), 0)
\end{equation}
where $S(n, j)$ is the stock price at the $j$th node of the final time step.

For a European option, the value at any node is simply the discounted expected value of the next step's nodes. For an American option, the holder has a choice at every time step either to hold the option or exercise it immediately. The value of exercising a put option immediately at node $(i, j)$ is $\max(c - S(i, j), 0)$. A rational holder will always choose the action that maximises the option's value. Therefore, the backward induction step becomes
\begin{equation}
V(i, j) = \max( [pV(i+1, j+1) + (1 - p)V(i+1, j)] e^{-\rho \Delta t}, \max(c - S(i, j), 0) )
\end{equation}
This means the value at any node is the greater of its hold value (the discounted future expectation) and its exercise value (the intrinsic value if exercised immediately).

In [37]:
import numpy as np

def black_scholes_call_price(S0, c, t0, rho, sigma):
    '''
    Calculates the Black-Scholes price for a European call option.
    '''
    d1 = (np.log(S0 / c) + (rho + 0.5 * sigma**2) * t0) / (sigma * np.sqrt(t0))
    d2 = (np.log(S0 / c) + (rho - 0.5 * sigma**2) * t0) / (sigma * np.sqrt(t0))
    price = S0 * norm.cdf(d1) - c * np.exp(-rho * t0) * norm.cdf(d2)
    return price

def american_put_price(S0, c, t0, rho, sigma, n):
    '''
    Calculates the approximate price of an American put option
    using the binomial tree method.
    '''
    # Calculate parameters
    mu = rho - 0.5 * sigma**2
    dt = t0 / n
    g = np.sqrt(sigma**2 * dt + (mu * dt)**2)
    p = 0.5 * (1 + (mu * dt) / g)
    discount_factor = np.exp(-rho * dt)

    # Initialize arrays for option values
    V_next = np.zeros(n + 1)

    # Set boundary conditions at expiry (t=t0) for a PUT option
    for j in range(n + 1):
        stock_price_at_expiry = S0 * np.exp((2 * j - n) * g)
        V_next[j] = max(c - stock_price_at_expiry, 0)

    # Perform backward induction with early exercise check
    for i in range(n - 1, -1, -1):
        V_curr = np.zeros(i + 1)
        for j in range(i + 1):
            # Calculate the current stock price at this node
            stock_price_curr = S0 * np.exp((2 * j - i) * g)

            # Calculate the value of holding the option
            hold_value = (p * V_next[j + 1] + (1 - p) * V_next[j]) * discount_factor

            # Calculate the value of exercising the option now
            exercise_value = max(c - stock_price_curr, 0)

            # The option's value is the maximum of holding or exercising
            V_curr[j] = max(hold_value, exercise_value)

        V_next = V_curr

    return V_next[0]

c = 40.0
s0_values = [52, 100, 107]
sigma = 0.5
rho = 0.035
t0_values = [2, 3]
n_values_to_test = [27, 101]
print("Approximate Price of American Put Option")
header = f"{'S₀':<8} | {'t₀':<5} | {'n':<5} | {'Approx Put Price':<15}"
print(header)
print("-" * len(header))

for s0 in s0_values:
    for t0 in t0_values:
        for n in n_values_to_test:
            price = american_put_price(s0, c, t0, rho, sigma, n)
            print(f"{s0:<8} | {t0:<5} | {n:<5} | ${price:<13.2f}")
        print("-" * len(header))

Approximate Price of American Put Option
S₀       | t₀    | n     | Approx Put Price
-------------------------------------------
52       | 2     | 27    | $6.53         
52       | 2     | 101   | $6.46         
-------------------------------------------
52       | 3     | 27    | $8.38         
52       | 3     | 101   | $8.30         
-------------------------------------------
100      | 2     | 27    | $1.55         
100      | 2     | 101   | $1.54         
-------------------------------------------
100      | 3     | 27    | $2.95         
100      | 3     | 101   | $2.95         
-------------------------------------------
107      | 2     | 27    | $1.26         
107      | 2     | 101   | $1.31         
-------------------------------------------
107      | 3     | 27    | $2.63         
107      | 3     | 101   | $2.58         
-------------------------------------------


Unlike the oscillatory convergence observed for the European call option, the price of the American put option exhibits a much more stable and monotonic convergence.

The key difference is the availability of early exercise. The max function applied at every node in the backward induction step acts to smooth the approximation. It prevents the price from undershooting its intrinsic value at any point. The option's calculated value at any node is floored by its immediate exercise value. This dampens the oscillations that were caused by the specific alignment of the final nodes relative to the strike price in the European option case.

As $n$ increases, the grid of possible stock prices becomes finer, allowing for a more accurate representation of the optimal exercise boundary. Because the early exercise decision is re-evaluated at each of these finer steps, the overall price converges smoothly and monotonically from below towards the true, continuous-time value.


---

Suppose that $f_n$ is the approximation to the option price, and we wish to find the limiting value of $f_n$ as $n \to \infty$. One method is Richardson extrapolation which assumes that $f_n$ may be approximated by a polynomial in $1/n$
\begin{equation}
    f_n \approx g_0 + g_1n^{-1} + g_2n^{-2} + \cdots + g_sn^{−s}.
\end{equation}
The limiting value of $f_n$ is then approximated by $g_0$. One way to achieve this is as follows. Let $n_m = r^mn_0$ and calculate $f_n$ at $n = n_0, \dots, n_s$. Set
\begin{equation}
    a_{m,0} = f_{n_m} for m = 0, \dots, s
\end{equation}
and recursively calculate
\begin{equation}
    a_{m,i} = a_{m,i−1} + \frac{a_{m,i−1} − a_{m−1,i−1}}{r^i − 1} \quad \text{for} \quad m = i, \dots, s \quad \text{and} \quad i = 1, \dots, s.
\end{equation}
Then $a_{s,s}$ is taken as the approximation for $g_0$.

In [38]:
import numpy as np
from scipy.stats import norm

def bernoulli_call_price(S0, c, t0, rho, sigma, n):
    '''
    Calculates the approximate price of a European call option.
    '''
    if n == 0: return max(S0 - c, 0) # Handle n=0 case
    mu = rho - 0.5 * sigma**2
    dt = t0 / n
    g = np.sqrt(sigma**2 * dt + (mu * dt)**2)
    p = 0.5 * (1 + (mu * dt) / g)
    V_next = np.zeros(n + 1)
    for j in range(n + 1):
        V_next[j] = max(S0 * np.exp((2 * j - n) * g) - c, 0)
    discount_factor = np.exp(-rho * dt)
    for i in range(n - 1, -1, -1):
        V_curr = np.zeros(i + 1)
        for j in range(i + 1):
            V_curr[j] = (p * V_next[j + 1] + (1 - p) * V_next[j]) * discount_factor
        V_next = V_curr
    return V_next[0]

def richardson_extrapolation(price_func, s, r, n0, S0, c, t0, rho, sigma):
    '''
    Performs Richardson extrapolation to estimate the limiting value of a price function.
    Args:
        price_func: The function to calculate the price for a given n (our f_n).
        s: The degree of the polynomial approximation.
        r: The scaling factor for n.
        n0: The initial number of steps.
        ... pricing parameters ...
    Returns:
        The extrapolated price (approximation for g0).
    '''
    # Initialize the table `a`
    a = np.zeros((s + 1, s + 1))

    # Calculate the initial column a_{m,0}
    for m in range(s + 1):
        nm = int(n0 * (r**m))
        a[m, 0] = price_func(S0, c, t0, rho, sigma, nm)

    # Recursively calculate the rest of the table
    for i in range(1, s + 1):
        for m in range(i, s + 1):
            numerator = a[m, i-1] - a[m-1, i-1]
            denominator = r**i - 1
            a[m, i] = a[m, i-1] + numerator / denominator

    # The result is the bottom-right element
    return a[s, s]

S0, c, t0, sigma, rho = 50.0, 50.0, 2.0, 0.5, 0.035
true_price = black_scholes_call_price(S0, c, t0, rho, sigma)
r = 2
s_values = [2, 3, 4]
n0_values = [10, 11]

print(f"True Black-Scholes Price: {true_price:.8f}")
for n0 in n0_values:
    print(f"\n--- Starting with n0 = {n0} ({'Even' if n0 % 2 == 0 else 'Odd'}) ---")

    for s in s_values:
        # The largest n used in this extrapolation
        n_large = n0 * (r**s)

        # Price from a single large-n calculation
        price_large_n = bernoulli_call_price(S0, c, t0, rho, sigma, n_large)
        error_large_n = abs(price_large_n - true_price)

        # Price from Richardson extrapolation
        extrapolated_price = richardson_extrapolation(bernoulli_call_price, s, r, n0, S0, c, t0, rho, sigma)
        error_extrapolation = abs(extrapolated_price - true_price)

        print(f"  s = {s}:")
        print(f"    Single Large n ({n_large}): Price = {price_large_n:.8f}, Error = {error_large_n:.8f}")
        print(f"    Extrapolation:           Price = {extrapolated_price:.8f}, Error = {error_extrapolation:.8f}")

True Black-Scholes Price: 15.10208773

--- Starting with n0 = 10 (Even) ---
  s = 2:
    Single Large n (40): Price = 15.03664011, Error = 0.06544763
    Extrapolation:           Price = 15.10215111, Error = 0.00006337
  s = 3:
    Single Large n (80): Price = 15.06931788, Error = 0.03276986
    Extrapolation:           Price = 15.10208810, Error = 0.00000036
  s = 4:
    Single Large n (160): Price = 15.08569169, Error = 0.01639605
    Extrapolation:           Price = 15.10208773, Error = 0.00000001

--- Starting with n0 = 11 (Odd) ---
  s = 2:
    Single Large n (44): Price = 15.04257439, Error = 0.05951335
    Extrapolation:           Price = 15.29513853, Error = 0.19305080
  s = 3:
    Single Large n (88): Price = 15.07229324, Error = 0.02979449
    Extrapolation:           Price = 15.07451615, Error = 0.02757159
  s = 4:
    Single Large n (176): Price = 15.08718133, Error = 0.01490640
    Extrapolation:           Price = 15.10392585, Error = 0.00183812


The extrapolation procedure is dramatically more accurate than just calculating the price for a single, suitably large value of $n$. The error in the binomial approximation for a European option has a known asymptotic expansion, which for a smooth payoff is
\begin{equation}
    f_n = g_0 + g_1/n + g_2/n^2 + O(1/n^3)
\end{equation}
The at-the-money case is not smooth due to the spike in the payoff function $\max(S-c, 0)$ at $S=c$. The odd-even oscillations introduce an additional complexity. The error expansion contains both even and odd powers of $1/\sqrt{n}$ and $1/n$. A more complete, though simplified, view of the error is
\begin{equation}
    E(n) = f_n - g_0 \approx c_1/n + (c_2 (-1)^2) / n + \cdots
\end{equation}
The $(-1)^n$ term mathematically represents the odd-even oscillations. The Richardson extrapolation which assumes a polynomial in $1/n$, works by systematically cancelling out the $c_1/n$, $c_2/n^2$, etc., terms. Even though it doesn't explicitly account for the oscillatory term, it is still effective as the oscillatory term's magnitude also decreases with $n$, and the process of combining results from different $n$ values helps to average out and reduce this oscillatory error.

The final extrapolated error for a given $s$ is very similar for both odd and even $n_0$ but is still slightly higher for odd $n_0$.

Explanation of the Dependence: The choice of n₀ determines the path the approximation takes towards the final value.
*   If $n_0$ is even, then all subsequent $n$ values $n_0 r^m$ in the sequence will also be even. The initial calculations $a_{m,0}$ will all come from the undershooting side of the convergence graph.
*   If $n_0$ is odd, then all subsequent $n$ values will also be odd, and the initial calculations will all come from the overshooting side instead.

The extrapolation procedure is robust enough to work with either set of systematically biased inputs as it approximates the limit of a sequence, of which both the odd and even subsequences will converge to. The minor differences in the final error are due to higher-order error terms that are not perfectly captured by the model.


---

The Bernoulli method may be refined by replacing the Bernoulli distribution between times $it_0/n$ and $(i + 1)t_0/n$ by a binomial distribution taking $k + 1$ equally-spaced values with mean and variance chosen to match those of the Brownian motion.

In this model, the increment in the log-price over a time step $\Delta t = t_0/n$ is modeled by a random variable $X$. This $X$ is the sum of $k$ independent Bernoulli trials, each with a step size of $g$ and probability of an upward move $p$.

The distribution for the number of up moves in $k$ trials is $\mathop{Binomial}(k, p)$. Let $J$ be this number. The mean and variance of $J$ are $E[J] = kp$ and $Var(J) = kp(1-p)$. The random variable $X$ can be written as
\begin{equation}
    X = (J - (k - J))g = (2J - k)g.
\end{equation}
Set the mean and variance of $X$ to match the underlying Brownian motion over the interval $\Delta t$
\begin{equation}
    E[X] = \mu\Delta t, \\
    Var(X) = \sigma^2\Delta t.
\end{equation}
Using the properties of expectation and variance,
\begin{equation}
    E[X] = kg(2p - 1), \\
    Var(X) = 4g²kp(1-p).
\end{equation}
Now we set up our system of equations,
\begin{equation}
    kg(2p - 1) = \mu\Delta t, \\
    4g^2kp(1-p) = \sigma^2\Delta t.
\end{equation}
Solving this system for $g$ and $p$, we get
\begin{equation}
    g = \sqrt{\sigma^2\Delta t/k + (\mu\Delta t / k)^2}, \\
    p = \frac{1}{2}(1 + \mu\Delta t / (kg)).
\end{equation}


In [39]:
import numpy as np
from scipy.special import comb

def binomial_pricer(S0, c, t0, rho, sigma, n, k, option_type='call'):
    '''
    Prices an option using the generalized binomial tree model.
    '''
    # Calculate model parameters
    mu = rho - 0.5 * sigma**2
    dt = t0 / n

    # Handle the case where the term inside the square root is negative
    # This can happen for very high mu and low sigma, requires dt to be small enough
    variance_term = sigma**2 * dt / k
    mean_term_sq = (mu * dt / k)**2
    if variance_term + mean_term_sq < 0:
        raise ValueError("Invalid parameters: negative value in sqrt for g.")

    g = np.sqrt(variance_term + mean_term_sq)
    p = 0.5 * (1 + (mu * dt) / (k * g))

    if not (0 <= p <= 1):
        # This check is important. It ensures the model is stable.
        # It requires n to be large enough relative to k, sigma, and rho.
        print(f"Warning: p = {p:.4f} is outside [0, 1]. Results may be unstable.")

    discount_factor = np.exp(-rho * dt)

    # Calculate binomial probabilities P(j) for j=0..k up moves
    binomial_probs = [comb(k, j) * (p**j) * ((1-p)**(k-j)) for j in range(k + 1)]

    # Initialize the tree at expiry
    # The max number of up steps after n periods of k trials is n*k
    num_final_nodes = n * k + 1
    V_next = np.zeros(num_final_nodes)

    for j in range(num_final_nodes):
        # j here is the total number of upward 'g' steps
        stock_price = S0 * np.exp((j - (n*k - j)) * g) # Equivalent to S0 * exp((2j - n*k)*g)
        if option_type == 'call':
            V_next[j] = max(stock_price - c, 0)
        else: # put
            V_next[j] = max(c - stock_price, 0)

    # Backward induction
    for i in range(n - 1, -1, -1):
        num_curr_nodes = i * k + 1
        V_curr = np.zeros(num_curr_nodes)
        for j in range(num_curr_nodes):
            # Calculate the expected future value by summing over k+1 possibilities
            expected_value = 0
            for l in range(k + 1):
                # A node (i,j) can go to node (i+1, j+l)
                expected_value += binomial_probs[l] * V_next[j + l]

            hold_value = expected_value * discount_factor

            if option_type == 'american_put':
                stock_price = S0 * np.exp((j - (i*k - j)) * g)
                exercise_value = max(c - stock_price, 0)
                V_curr[j] = max(hold_value, exercise_value)
            else: # European Call
                V_curr[j] = hold_value
        V_next = V_curr

    return V_next[0]

# --- Main part of the script ---
S0, c, t0, sigma, rho = 50.0, 50.0, 2.0, 0.5, 0.035
true_call_price = 12.82286
n = 50 # A moderately large n

print("--- Comparison of Binomial Model (k>1) vs. Bernoulli (k=1) ---")
print(f"Parameters: S0={S0}, c={c}, t0={t0}, n={n}\n")
print(f"{'Option Type':<15} | {'k':<5} | {'Approximate Price':<20}")

print("-" * 50)
for k in [1, 2, 5, 10]:
    price = binomial_pricer(S0, c, t0, rho, sigma, n, k, 'call')
    print(f"{'European Call':<15} | {k:<5} | ${price:<20.4f}")

print("-" * 50)
for k in [1, 2, 5, 10]:
    # Using different S0 for a more interesting put price
    price = binomial_pricer(45, c, t0, rho, sigma, n, k, 'american_put')
    print(f"{'American Put':<15} | {k:<5} | ${price:<20.4f}")

--- Comparison of Binomial Model (k>1) vs. Bernoulli (k=1) ---
Parameters: S0=50.0, c=50.0, t0=2.0, n=50

Option Type     | k     | Approximate Price   
--------------------------------------------------
European Call   | 1     | $15.0497             
European Call   | 2     | $15.0759             
European Call   | 5     | $15.0916             
European Call   | 10    | $15.0968             
--------------------------------------------------
American Put    | 1     | $14.0745             
American Put    | 2     | $14.0236             
American Put    | 5     | $14.0102             
American Put    | 10    | $14.0094             


For both European and American optionss, increasing $k$ leads to a faster convergence towards the true option price. A higher $k$ creates a more complex tree that better approximates the continuous distribution of the underlying Brownian motion for the same number of time steps.

The benefit of increasing $k$ is more pronounced for the American put option. The value of an American option is highly dependent on accurately defining the optimal exercise boundary. A larger $k1$ creates a much denser grid of possible prices at each time step which allows for a much more precise estimation of the exercise boundary's location, leading to a more accurate valuation of early exercise. The European option has no such feature, so while it also benefits from the better distributional approximation, the effect is less critical.

We now analyse complexity. The outer loop runs $n$ times for each time step. The middle loop runs $O(ik)$ times, where $i$ goes up to $n$ which is $O(nk)$.
The innermost loop runs $k$ times. Combining these gives a rough time complexity of $O(n^2k^2)$ The complexity is quadratic in $n$ and $k$ which means that this is significantly more computationally expensive than the $k=1$ Bernoulli approximation, but it can achieve higher accuracy for a smaller $n$.